In [ ]:
# --- 1. 라이브러리 및 모듈 임포트 ---
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 데이터 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 머신러닝 모델 및 평가 도구
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import joblib

# 모델 해석 및 시각화 (SHAP)
# !pip install shap
import shap
from IPython.display import display

# 하이퍼파라미터 최적화 라이브러리
# !pip install optuna
import optuna

# 시스템 및 경로 관련 라이브러리
import sys
import os
from datetime import datetime
import warnings

# 경고 메시지 무시
warnings.filterwarnings('ignore')

# 한글 폰트 설정
import matplotlib.font_manager as fm
try:
    font_path = '../../font/NanumFont/NanumGothic.ttf'
    if os.path.exists(font_path):
        fe = fm.FontEntry(fname=font_path, name='NanumGothic')
        fm.fontManager.ttflist.insert(0, fe)
        plt.rcParams.update({'font.size': 12, 'font.family': 'NanumGothic'})
    else:
        print("나눔고딕 폰트를 찾을 수 없어 기본 폰트로 설정됩니다.")
except Exception as e:
    print(f"폰트 설정 중 오류 발생: {e}")
    pass


# --- 2. 경로 설정 및 커스텀 로거 임포트 ---
try:
    src_path = os.path.abspath(os.path.join(os.getcwd(), "../../src/log"))
    sys.path.insert(0, src_path)
    from logger import Logger
    print("Logger 모듈 로드 성공.")
except ImportError:
    print("[오류] Logger 모듈을 찾을 수 없습니다.")
    class Logger:
        def __init__(self, *args, **kwargs): pass
        def write(self, message, **kwargs): print(message)
        def start_redirect(self): pass
        def stop_redirect(self): pass
        def close(self): pass


# --- 3. 로거 및 경로 초기화 ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_DIR = '../../data/logs/price_prediction_3_logs'
LOG_FILENAME = f"price_prediction_{timestamp}.log"
LOG_PATH = os.path.join(LOG_DIR, LOG_FILENAME)
os.makedirs(LOG_DIR, exist_ok=True)
logger = Logger(log_path=LOG_PATH)

TRAIN_PATH = '../../data/processed/cleaned_data/train_clean.csv'
TEST_PATH = '../../data/processed/cleaned_data/test_clean.csv'

SUBMISSION_DIR = '../../data/processed/submissions'
SUBMISSION_FILENAME = 'price_prediction_3_submission.csv'
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, SUBMISSION_FILENAME)
os.makedirs(SUBMISSION_DIR, exist_ok=True)

MODEL_DIR = '../../model'
MODEL_FILENAME = 'price_prediction_3_model.pkl'
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)
os.makedirs(MODEL_DIR, exist_ok=True)

logger.write("="*50)
logger.write(">> 아파트 가격 예측 모델링 시작")
logger.write(">> [1단계 완료] 라이브러리 및 모듈 임포트 성공")
logger.write(">> [2단계 완료] 경로 설정 및 커스텀 로거 임포트 성공")
logger.write(">> [3단계 완료] 로거 및 경로 초기화 성공")

# --- 4. 데이터 로드 ---
logger.write(">> [4단계 시작] 정제된 Train/Test 데이터를 로드합니다...")
try:
    train_df_clean = pd.read_csv(TRAIN_PATH)
    test_df_clean = pd.read_csv(TEST_PATH)
    
    train_df_clean['isTest'] = 0
    test_df_clean['isTest'] = 1
    
    df = pd.concat([train_df_clean, test_df_clean])
    logger.write(">> 데이터 로드 및 병합 완료.")
except FileNotFoundError as e:
    logger.write(f">> [오류] 데이터 파일을 찾을 수 없습니다: {e}", print_error=True)
    df = None


# --- 5. 피처 엔지니어링 및 데이터 분리 (성능 개선 버전) ---

# --- 5-1. 타겟 변환 (로그 변환) ---
if 'df' in locals() and df is not None:
    logger.write(">> [5-1단계 시작] Target 변수 로그 변환을 시작합니다...")
    train_df = df[df['isTest'] == 0].copy()
    train_df['target'] = np.log1p(train_df['target'])
    df.loc[df['isTest']==0, 'target'] = train_df['target']
    logger.write(">> Target 변수 로그 변환 완료.")


# --- 5-2. 고급 피처 엔지니어링 ---
if 'df' in locals() and df is not None:
    try:
        logger.write(">> [5-2단계 시작] 고급 피처 엔지니어링 및 데이터 분리를 시작합니다...")
        
        # 1. Train/Test 데이터 분리 (가장 먼저 수행)
        train_df = df[df['isTest'] == 0].copy()
        test_df = df[df['isTest'] == 1].copy()
        logger.write(">> 1. Train/Test 데이터 분리 완료.")

        # 2. 시간 관련 파생변수
        logger.write(">> 2. 시간 관련 피처 생성 중...")
        current_year = datetime.now().year
        for temp_df in [train_df, test_df]:
            temp_df['계약월_sin'] = np.sin(2 * np.pi * temp_df['계약월']/12)
            temp_df['계약월_cos'] = np.cos(2 * np.pi * temp_df['계약월']/12)
            temp_df['아파트나이'] = current_year - temp_df['연식']
        logger.write(">> '계약월_sin/cos', '아파트나이' 피처 생성 완료.")

        # 3. 면적당 가격(평당가 개념) 파생변수 (데이터 누수 방지)
        logger.write(">> 3. 면적당 가격 관련 피처 생성 중...")
        train_df['면적당가격'] = train_df['target'] / train_df['전용면적']
        
        dong_price_stats = train_df.groupby('법정동')['면적당가격'].agg(['mean', 'std']).reset_index()
        dong_price_stats.columns = ['법정동', '동별_평균면적당가격', '동별_면적당가격편차']
        gu_price_stats = train_df.groupby('자치구')['면적당가격'].agg(['mean', 'std']).reset_index()
        gu_price_stats.columns = ['자치구', '구별_평균면적당가격', '구별_면적당가격편차']
        
        train_df = pd.merge(train_df, dong_price_stats, on='법정동', how='left')
        test_df = pd.merge(test_df, dong_price_stats, on='법정동', how='left')
        train_df = pd.merge(train_df, gu_price_stats, on='자치구', how='left')
        test_df = pd.merge(test_df, gu_price_stats, on='자치구', how='left')
        logger.write(">> '동별/구별 면적당가격' 피처 생성 및 병합 완료.")
        
        # 4. 상호작용 특성 강화
        logger.write(">> 4. 상호작용 특성 생성 중...")
        for temp_df in [train_df, test_df]:
            temp_df['면적_x_나이'] = temp_df['전용면적'] * temp_df['아파트나이']
            temp_df['면적_x_층'] = temp_df['전용면적'] * temp_df['층']
            temp_df['강남_x_면적'] = temp_df['강남3구여부'] * temp_df['전용면적']
        logger.write(">> 상호작용 특성 생성 완료.")
        
        # 5. 테스트 데이터의 결측치(NA) 처리
        logger.write(">> 5. 테스트 데이터의 결측치(NA) 처리 중...")
        fill_na_cols = ['동별_평균면적당가격', '동별_면적당가격편차', '구별_평균면적당가격', '구별_면적당가격편차']
        mean_vals = train_df[fill_na_cols].mean()
        test_df.fillna(mean_vals, inplace=True)
        logger.write(">> 테스트 데이터 결측치 처리 완료.")
        
        # 6. 피처(X)와 타겟(y) 최종 정의
        logger.write(">> 6. 학습에 사용할 피처(X)와 타겟(y) 정의 중...")
        features = [col for col in train_df.columns if col not in ['target', 'id', '아파트이름', 'isTest', '면적당가격']]
        
        X_train_raw = train_df[features]
        y_train = train_df['target']
        X_test_raw = test_df[features]
        logger.write(">> 피처/타겟 정의 완료.")

        # 7. 범주형 피처 처리 (Label Encoding)
        logger.write(">> 7. 범주형 피처 Label Encoding 처리 중...")
        categorical_features = X_train_raw.select_dtypes(include=['object']).columns.tolist()
        
        for col in categorical_features:
            le = LabelEncoder()
            all_vals = pd.concat([X_train_raw[col], X_test_raw[col]]).astype(str).unique()
            le.fit(all_vals)
            X_train_raw[col] = le.transform(X_train_raw[col].astype(str))
            X_test_raw[col] = le.transform(X_test_raw[col].astype(str))
            
        logger.write(">> [5단계 완료] 모든 피처 엔지니어링 및 데이터 분리 성공.")

    except Exception as e:
        logger.write(f">> [오류] 피처 엔지니어링 중 심각한 문제 발생: {e}", print_error=True)


# --- 6. 하이퍼파라미터 최적화 및 모델 학습 ---

# --- 6-1. (신규) 하이퍼파라미터 최적화 (Optuna) ---
def objective(trial, X, y):
    param = {
        'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000,
        'verbosity': -1, 'boosting_type': 'gbdt', 'seed': 42,
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

        model = lgb.LGBMRegressor(device='cuda', **param)
        model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  eval_metric='rmse',
                  callbacks=[lgb.early_stopping(50, verbose=False)])
        
        val_preds = model.predict(X_val_fold)
        rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
        rmse_scores.append(rmse)
        
    return np.mean(rmse_scores)

if 'X_train_raw' in locals():
    logger.write(">> [6-1단계 시작] Optuna로 하이퍼파라미터 최적화를 시작합니다...")
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_train_raw, y_train), n_trials=30) # 30회 시도
    
    best_params = study.best_params
    best_params.update({
        'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000,
        'verbosity': -1, 'seed': 42
    })
    logger.write(f">> Optuna 탐색 완료. 최적 파라미터: {best_params}")


# --- 6-2. 최종 모델 학습 (K-Fold) ---
if 'X_train_raw' in locals() and 'best_params' in locals():
    logger.start_redirect()
    try:
        logger.write(">> [6-2단계 시작] 최적 파라미터로 K-Fold 교차 검증 및 모델 학습을 시작합니다...")
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        oof_preds = np.zeros(X_train_raw.shape[0])
        test_preds = np.zeros(X_test_raw.shape[0])
        rmse_scores = []
        models = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_raw, y_train)):
            logger.write(f"--- [Fold {fold+1}/5] 학습 시작 ---")
            X_train_fold, X_val_fold = X_train_raw.iloc[train_idx], X_train_raw.iloc[val_idx]
            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model = lgb.LGBMRegressor(device='cuda', **best_params)
            model.fit(X_train_fold, y_train_fold,
                      eval_set=[(X_val_fold, y_val_fold)],
                      eval_metric='rmse',
                      callbacks=[lgb.early_stopping(100, verbose=False)])

            val_preds = model.predict(X_val_fold)
            oof_preds[val_idx] = val_preds
            
            rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
            rmse_scores.append(rmse)
            models.append(model)
            logger.write(f"✅ Fold {fold+1} Log-RMSE: {rmse:.4f}")
            
            test_preds += np.expm1(model.predict(X_test_raw)) / kf.get_n_splits()

        avg_rmse = np.mean(rmse_scores)
        logger.write("-------------------------------------------")
        logger.write(f"✅ 최종 CV 평균 Log-RMSE: {avg_rmse:.4f}")
        logger.write("-------------------------------------------")
        
        logger.write(">> [6-2단계 완료] 모든 모델 학습 및 평가 성공.")
    except Exception as e:
        logger.write(f">> [오류] 모델 학습 중 심각한 문제 발생: {e}", print_error=True)
    finally:
        logger.stop_redirect()


# --- 7. 최종 모델 결과 시각화 및 분석 ---
if 'models' in locals() and 'oof_preds' in locals():
    logger.write(">> [7단계 시작] 모델 결과 시각화 및 분석을 시작합니다...")
    try:
        # 1. 피처 중요도(Feature Importance) 시각화 (폴드별 평균)
        logger.write(">> 1. 피처 중요도 시각화 중...")
        feature_importances = pd.DataFrame()
        for i, model in enumerate(models):
            fold_importance = pd.DataFrame({
                'feature': X_train_raw.columns,
                'importance': model.feature_importances_,
                'fold': i + 1
            })
            feature_importances = pd.concat([feature_importances, fold_importance], axis=0)
        
        mean_importances = feature_importances.groupby('feature')['importance'].mean().sort_values(ascending=False)
        
        plt.figure(figsize=(12, 10))
        sns.barplot(x=mean_importances.head(20).values, y=mean_importances.head(20).index)
        plt.title('상위 20개 피처 중요도 (평균)', fontsize=16)
        plt.xlabel('중요도', fontsize=12)
        plt.ylabel('피처', fontsize=12)
        plt.grid(True, axis='x', linestyle='--')
        plt.tight_layout()
        plt.show()
        logger.write(">> 피처 중요도 시각화 완료.")

        # 2. 실제 값 vs OOF 예측 값 비교 (Out-of-Fold)
        logger.write(">> 2. 실제 값 vs OOF 예측 값 비교 시각화 중...")
        plt.figure(figsize=(10, 10))
        # ✨ 중요: 로그 변환된 값을 원래 스케일로 되돌려 시각화합니다.
        sns.scatterplot(x=np.expm1(y_train), y=np.expm1(oof_preds), alpha=0.3)
        plt.plot([np.expm1(y_train).min(), np.expm1(y_train).max()], [np.expm1(y_train).min(), np.expm1(y_train).max()], 'r--', lw=2, label='이상적인 예측선 (y=x)')
        plt.title('실제 값 vs OOF 예측 값 비교', fontsize=16)
        plt.xlabel('실제 값 (Actual)', fontsize=12)
        plt.ylabel('OOF 예측 값 (Out-of-Fold Predicted)', fontsize=12)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        logger.write(">> 실제 값 vs OOF 예측 값 비교 시각화 완료.")

        # 3. 잔차(Residuals) 분포 확인 (OOF 기반)
        logger.write(">> 3. 잔차 분포 확인 시각화 중...")
        residuals = np.expm1(y_train) - np.expm1(oof_preds)
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True, bins=50)
        plt.title('잔차(실제-예측) 분포 (OOF 기반)', fontsize=16)
        plt.xlabel('잔차 (Residuals)', fontsize=12)
        plt.ylabel('빈도 (Frequency)', fontsize=12)
        plt.axvline(x=0, color='red', linestyle='--')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        logger.write(">> 잔차 분포 확인 시각화 완료.")
        
        # 4. 학습 데이터와 테스트 예측 결과 분포 비교
        logger.write(">> 4. 학습/테스트 데이터 예측 분포 비교 시각화 중...")
        plt.figure(figsize=(10, 6))
        sns.kdeplot(np.expm1(y_train), label='학습 데이터 실제 값', color='blue', fill=True, alpha=0.5)
        sns.kdeplot(test_preds, label='테스트 데이터 예측 값', color='orange', fill=True, alpha=0.5)
        plt.title('학습 데이터와 테스트 예측의 분포 비교', fontsize=16)
        plt.xlabel('아파트 가격', fontsize=12)
        plt.ylabel('밀도 (Density)', fontsize=12)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        logger.write(">> 학습/테스트 데이터 예측 분포 비교 시각화 완료.")

        # 5. SHAP 분석 (첫 번째 폴드 모델 사용)
        logger.write(">> 5. SHAP 분석 시작 (계산에 시간이 다소 소요될 수 있습니다)...")
        explainer = shap.TreeExplainer(models[0])
        shap_sample = X_train_raw.sample(1000, random_state=42)
        shap_values = explainer.shap_values(shap_sample)

        logger.write(">> SHAP 요약 플롯 생성 중...")
        shap.summary_plot(shap_values, shap_sample, plot_type="dot", show=False)
        plt.title("SHAP 요약 플롯 (첫 번째 폴드 모델)", fontsize=16)
        plt.tight_layout()
        plt.show()
        logger.write(">> SHAP 분석 완료.")
        
        logger.write(">> [7단계 완료] 시각화 및 분석 완료.")

    except Exception as e:
        logger.write(f">> [오류] 시각화 및 분석 중 문제가 발생했습니다: {e}", print_error=True)


# --- 8. 최종 예측 및 제출 파일 생성 ---
if 'test_preds' in locals():
    logger.write(">> [8단계 시작] 최종 제출 파일 생성을 시작합니다...")
    try:
        # 최종 모델로 전체 데이터 학습 (나중에 다른 예측에 사용하기 위해 저장)
        logger.write(">> 전체 훈련 데이터로 최종 모델 학습 시작...")
        final_model = lgb.LGBMRegressor(device='cuda', **best_params)
        final_model.fit(X_train_raw, y_train)
        logger.write(">> 최종 모델 학습 완료.")
        
        # 모델 저장
        joblib.dump(final_model, MODEL_PATH)
        logger.write(f">> 모델 저장 완료: {MODEL_PATH}")

        # 제출 파일 생성
        logger.write(">> 'target' 컬럼만 포함된 제출 파일을 생성합니다.")
        submission_df = pd.DataFrame({'target': test_preds})
        submission_df['target'] = submission_df['target'].astype(int)
        
        submission_df.to_csv(SUBMISSION_PATH, index=False)
        logger.write(f">> 제출 파일 생성 완료: {SUBMISSION_PATH}")
        
    except Exception as e:
        logger.write(f">> [오류] 제출 파일 생성 중 문제 발생: {e}", print_error=True)
    
    logger.write(">> 모델링 종료")
    logger.write("="*50 + "\n")
    logger.close()
